In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import datetime
import os
from skimage.transform import rotate
import random
from tensorflow.python.keras.utils.data_utils import Sequence
from scipy.ndimage import zoom
from scipy.ndimage import shift
from skimage.transform import resize

In [57]:
#input variables
#target_path = "C:\\Users\\awe2\\DL_DES-master\\data\\sdss-galaxyzoo\\high_certainty\\sdss_metadata\\"

path = 'image_arrays_new_new_new\\'
validation_path = path + 'test'
training_path = path + 'training'

#model variables
batch_size = 32 #
epoch_number = 50
learning_rate = 1e-3 

params = {'dim': (64,64),
          'batch_size': batch_size,
          'n_classes': 2,
          'n_channels': 5,
          'shuffle': True}


#more parameters means more prone to overfitting, and I am 5/3 times worse on parameters compared to the paper I have
#based this on. (5 bands instead of 3) I need to find ways to add more regularization, or otherwise might try reducing my number
#of layers to reduce the number of parameters.

In [58]:
#https://stackoverflow.com/questions/37119071/scipy-rotate-and-zoom-an-image-without-changing-its-dimensions/48097478
def clipped_zoom(img, zoom_factor, **kwargs):

    h, w = img.shape[:2]

    # For multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # Zooming out
    if zoom_factor < 1:

        # Bounding box of the zoomed-out image within the output array
        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        # Zero-padding
        out = np.zeros_like(img)
        out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)

    # Zooming in
    elif zoom_factor > 1:

        # Bounding box of the zoomed-in region within the input array
        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple, **kwargs)

        # `out` might still be slightly larger than `img` due to rounding, so
        # trim off any extra pixels at the edges
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    # If zoom_factor == 1, just return the input array
    else:
        out = img
    return out

In [59]:
#https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
class DataGenerator(Sequence):

    def __init__(self, list_IDs, labels, batch_size=32, dim=(120,120), n_channels=3,
                 n_classes=2, shuffle=True):
     #   'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def on_epoch_end(self):
    #'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
    #'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
    # Initialization
    
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)
        X_resized = np.empty((self.batch_size, 120, 120, self.n_channels))
      
      # Generate data and perform augmentation
        for i, ID in enumerate(list_IDs_temp):
            
          # Store sample
            X[i,] = np.load('image_arrays/' + ID + '.npy')
                          
            #flip
            if random.random() > 0.5:
                X[i,] = np.flip(X[i,],0)
            if random.random() > 0.5:
                X[i,] = np.flip(X[i,],1)
            
            #shift
            if random.random() > 0.5 :
                X[i,] = shift(X[i,], (4,0,0), mode='nearest')
            elif random.random() > 0.5 :
                X[i,] = shift(X[i,], (-4,0,0), mode='nearest')
                              
            if random.random() > 0.5 :
                X[i,] = shift(X[i,], (0,4,0), mode='nearest')
            elif random.random() > 0.5 :
                X[i,] = shift(X[i,], (0,-4,0), mode='nearest')
          
            #zoom in/out
            zoom_factor = random.uniform(0.75,1.3)
            X[i,] = clipped_zoom(X[i,],zoom_factor)
            
            #rotate
            angle = 45*random.random()
            X[i,] = rotate(X[i,], angle=angle, mode='reflect')
            
            #resize
            X_resized[i,] = resize(X[i,], (120,120,5) )
            
            # Store class
            y[i] = self.labels[ID]
    
        if self.n_classes > 2:
            return X_resized, keras.utils.to_categorical(y, num_classes=self.n_classes)
        else:
            return X_resized, y

    def __len__(self):
    #'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
    #  'Generate one batch of data'
      # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

      # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

      # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

In [60]:
galaxyzoo = pd.read_csv("zoo2MainSpecz.csv", usecols=[8,15,21], nrows=10000)
Class = galaxyzoo["gz2class"].values
dictionary = {'A':int(2),'E':np.array([0]),'S':np.array([1])}
#resave using my dictionary
target = np.empty((len(Class)))
for i in range(len(Class)):
    target[i] = dictionary[Class[i][0]]
    
spiral_debiased = galaxyzoo["t01_smooth_or_features_a02_features_or_disk_debiased"].values
elliptical_debiased = galaxyzoo["t01_smooth_or_features_a01_smooth_debiased"].values

In [61]:
train_list = os.listdir(training_path)
for i,file in enumerate(train_list):
    train_list[i] = file.split('.')[0]
val_list = os.listdir(validation_path)
for i,file in enumerate(val_list):
    val_list[i] = file.split('.')[0]

partition = {'train':train_list,'validation':val_list}

labels = {}
for i in range(10000):
    name = 'array_number_{}'.format(i)
    labels.update({name:target[i]})

In [62]:
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

In [63]:
def create_model(learning_rate=learning_rate):
    
    model = keras.Sequential([])
    
    model.add(keras.layers.Conv2D(input_shape=(120,120,5),filters=32,kernel_size=6,padding='same',activation=tf.nn.relu))
    model.add(keras.layers.Dropout(rate=0.5))
    
    model.add(keras.layers.Conv2D(filters=64,kernel_size=5,padding='same',activation=tf.nn.relu))
    model.add(keras.layers.MaxPool2D(pool_size=2,))
    model.add(keras.layers.Dropout(rate=0.25)) #best = 0.25
    
    model.add(keras.layers.Conv2D(filters=128,kernel_size=2,padding='same',activation=tf.nn.relu))
    model.add(keras.layers.MaxPool2D(pool_size=2,))
    model.add(keras.layers.Dropout(rate=0.25)) #best = 0.25
    
    
    model.add(keras.layers.Conv2D(filters=128,kernel_size=3,padding='same',activation=tf.nn.relu))
    model.add(keras.layers.Dropout(rate=0.35)) #best = 0.35

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(units=64,activation=tf.nn.relu))
    model.add(keras.layers.Dropout(rate=0.5))
    model.add(keras.layers.Dense(units=1,activation=tf.nn.sigmoid)) #tf.nn.softmax for categorical, sigmoid for binary
    
    adam = keras.optimizers.Adam(lr=learning_rate)
    model.compile(optimizer=adam, loss='binary_crossentropy',metrics=['accuracy']) 
    return(model)

In [64]:
keras.backend.clear_session()
model = create_model(learning_rate = learning_rate)

In [65]:
model.load_weights("models\\5-band-CNN-correct-filters-little-more.hdf5")

In [66]:
steps_to_take = int(len(train_list)/batch_size)
val_steps_to_take = int(len(val_list)/batch_size)
                #typically be equal to the number of unique samples if your dataset
                #divided by the batch size.
print(steps_to_take)

79


In [67]:
hist2 = model.evaluate_generator(generator=validation_generator,
                    steps=val_steps_to_take, 
                    verbose=1)

230/230 [==============================] - 333s 1s/step - loss: 0.6756 - acc: 0.5924


In [68]:
model.load_weights("models\\5-band-CNN-correct-filters-little-more.hdf5")

In [69]:
hist = model.fit_generator(generator=training_generator,
                    steps_per_epoch=steps_to_take, 
                    epochs=3,
                    verbose=1)

Epoch 1/3


C:\Users\awe2\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\ndimage\interpolation.py:605: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)
C:\Users\awe2\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\awe2\AppData\Local\Continuum\anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


79/79 [==============================] - 120s 2s/step - loss: 0.6815 - acc: 0.5811
Epoch 2/3
79/79 [==============================] - 72s 911ms/step - loss: 0.6792 - acc: 0.5767
Epoch 3/3
79/79 [==============================] - 73s 920ms/step - loss: 0.6764 - acc: 0.5985


In [70]:
hist1 = model.evaluate_generator(generator=validation_generator,
                    steps=val_steps_to_take, 
                    verbose=1)

230/230 [==============================] - 206s 897ms/step - loss: 0.6735 - acc: 0.5924


In [71]:
model.load_weights("models\\5-band-CNN-correct-filters-little-more.hdf5")

In [72]:
hist4 = model.fit_generator(generator=training_generator,
                    steps_per_epoch=steps_to_take, 
                    epochs=20,
                    verbose=1)

Epoch 1/20
79/79 [==============================] - 72s 916ms/step - loss: 0.6832 - acc: 0.5807
Epoch 2/20
79/79 [==============================] - 72s 910ms/step - loss: 0.6795 - acc: 0.5866
Epoch 3/20
79/79 [==============================] - 72s 907ms/step - loss: 0.6802 - acc: 0.5748
Epoch 4/20
79/79 [==============================] - 72s 905ms/step - loss: 0.6707 - acc: 0.5941
Epoch 5/20
79/79 [==============================] - 72s 913ms/step - loss: 0.6629 - acc: 0.6290
Epoch 6/20
79/79 [==============================] - 72s 916ms/step - loss: 0.6323 - acc: 0.6487
Epoch 7/20
79/79 [==============================] - 72s 912ms/step - loss: 0.5626 - acc: 0.7409
Epoch 8/20
79/79 [==============================] - 72s 912ms/step - loss: 0.5347 - acc: 0.7694
Epoch 9/20
79/79 [==============================] - 73s 919ms/step - loss: 0.4885 - acc: 0.7805
Epoch 10/20
79/79 [==============================] - 72s 916ms/step - loss: 0.4947 - acc: 0.7789
Epoch 11/20
79/79 [====================

In [73]:
hist5 = model.evaluate_generator(generator=validation_generator,
                    steps=val_steps_to_take, 
                    verbose=1)

230/230 [==============================] - 207s 900ms/step - loss: 0.4380 - acc: 0.7970


In [77]:
model.load_weights("models\\5-band-CNN-correct-filters-little-more.hdf5")

In [78]:
def lr_schedule(epoch,lr):

#Returns a custom learning rate that decreases as epochs progress.
    if epoch > 13:
        lr = 1e-4
    if epoch > 7:
        lr = 1e-5
    if epoch > 25:
        lr = 1e-6
    if epoch > 35:
        lr = 1e-7

    tf.summary.scalar('learning_rate', tensor=lr)
    return lr

lr_callback = keras.callbacks.LearningRateScheduler(lr_schedule)

In [79]:
hist4 = model.fit_generator(generator=training_generator,
                    steps_per_epoch=steps_to_take, 
                    epochs=50,
                    verbose=1,
                    callbacks=[lr_callback])

Epoch 1/50
79/79 [==============================] - 79s 1s/step - loss: 0.6865 - acc: 0.5799
Epoch 2/50
79/79 [==============================] - 74s 933ms/step - loss: 0.6802 - acc: 0.5819
Epoch 3/50
79/79 [==============================] - 73s 928ms/step - loss: 0.6631 - acc: 0.5965
Epoch 4/50
79/79 [==============================] - 73s 923ms/step - loss: 0.5813 - acc: 0.7049
Epoch 5/50
79/79 [==============================] - 74s 934ms/step - loss: 0.5281 - acc: 0.7504
Epoch 6/50
79/79 [==============================] - 74s 934ms/step - loss: 0.4714 - acc: 0.7809
Epoch 7/50
79/79 [==============================] - 72s 915ms/step - loss: 0.4747 - acc: 0.7686
Epoch 8/50
79/79 [==============================] - 72s 911ms/step - loss: 0.4565 - acc: 0.7959
Epoch 9/50
79/79 [==============================] - 72s 909ms/step - loss: 0.4232 - acc: 0.8058
Epoch 10/50
79/79 [==============================] - 72s 908ms/step - loss: 0.3944 - acc: 0.8323
Epoch 11/50
79/79 [=======================

In [80]:
hist5 = model.evaluate_generator(generator=validation_generator,
                    steps=val_steps_to_take, 
                    verbose=1)

230/230 [==============================] - 253s 1s/step - loss: 0.4024 - acc: 0.8262
